Load python packages

In [ ]:
# I'm sure loads of these are unnecessary!

import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from pathlib import Path
import xarray as xr
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
import matplotlib.colors as colors
import geopandas as gpd
from urllib.request import urlretrieve
import requests
from glob import glob

In [ ]:
parent_dir    = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_dir      = os.path.join(parent_dir, 'data')
satellite_dir = os.path.join(data_dir, 'satellite')
output_dir    = os.path.join(parent_dir, 'Output/sat_plot')
NEODASS_dir   = os.path.join(satellite_dir, 'NEODASS')

# Define the path to the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Check if 'data' folder exists in the parent directory
data_dir = os.path.join(parent_dir, 'data')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print("'data' folder created in the parent directory.")
else:
    print("'data' folder already exists in the parent directory.")
# Check if 'floats' directory exists inside 'data' folder
floats_dir = os.path.join(data_dir, 'Floats')
if not os.path.exists(floats_dir):
    os.makedirs(floats_dir)
    print("'floats' directory created inside 'data' folder.")
else:
    print("'floats' directory already exists inside 'data' folder.")
# Check if 'gliders' directory exists inside 'data' folder
gliders_dir = os.path.join(data_dir, 'Gliders')
if not os.path.exists(gliders_dir):
    os.makedirs(gliders_dir)
    print("'gliders' directory created inside 'data' folder.")
else:
    print("'gliders' directory already exists inside 'data' folder.")

#Products come from satellite_data_download.ipynb
bbp_data     = xr.open_dataset(os.path.join(satellite_dir, 'CMEMS_Iceland_Basin_BBP_April-May2024.nc'))
chla_data    = xr.open_dataset(os.path.join(satellite_dir, 'CMEMS_Iceland_Basin_CHLA_April-May2024.nc'))
#current_data = xr.open_dataset(os.path.join(satellite_dir, 'CMEMS_Iceland_Basin_SSH_April-May2024.nc'))
current_data = xr.open_dataset(os.path.join(satellite_dir, 'CMEMS_Iceland_Basin_SSC_April-May2024.nc'))

#Chla colorbar 
color = 'YlGnBu_r'

min_lon = -35
max_lon = -5
min_lat = 55
max_lat = 66

In [ ]:
last_date = chla_data.time[-2:]
# Filter the dataset to only include the last date
chla_data = chla_data.sel(time=last_date)

last_date = current_data.time[-2:]
# Filter the dataset to only include the last date
current_data = current_data.sel(time=last_date)

last_date = bbp_data.time[-2:]
# Filter the dataset to only include the last date
bbp_data = bbp_data.sel(time=last_date)


In [ ]:
display(chla_data)

In [ ]:
wmo_list = [4903532, 1902637]
#Float 1 = test float in the Icelandic Bassin
float_1_url = 'https://data-argo.ifremer.fr/dac/aoml/4903532/4903532_Sprof.nc'
#Float 2 = test float on Custard with glider next to it
float_2_url = 'https://data-argo.ifremer.fr/dac/coriolis/1902637/1902637_Sprof.nc'

#List the floats
floats_url = [float_1_url, float_2_url]

#Assign the local float directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
floats_dir = os.path.join(parent_dir, 'Data/Floats')

#Create floats filename
floats_filenames = []
for i in floats_url:
    filename = floats_dir + '/' + i.rsplit('/', 1)[1]
    floats_filenames.append(filename)

for url, filename in zip(floats_url, floats_filenames):
    urlretrieve(url, filename)

In [ ]:
#We will loop through the float files, we create two dataframe. One with all the positions from both floats, and one with only the last positions from both floats.
#Since we know plot the last position of the float at the date of the plot, the second one is not needed.

#Create empty df
position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})
last_position_df = pd.DataFrame({'PROF_NUM' : str(), 'LONGITUDE' : [], 'LATITUDE' : [], 'float' : int()})

for file, wmo in zip(floats_filenames, wmo_list):
    dat = xr.open_dataset(file)
    dat = dat.rename({'CYCLE_NUMBER':'PROF_NUM'}).swap_dims({'N_PROF':'PROF_NUM'})
    temp_df = dat[['LONGITUDE', 'LATITUDE', 'JULD']].to_dataframe().reset_index()
    temp_df['float'] = wmo
    last_temp_df = temp_df[temp_df['JULD'] == max(temp_df['JULD'])]

    last_position_df = pd.concat([last_position_df, last_temp_df], ignore_index=True)
    position_df = pd.concat([position_df, temp_df], ignore_index=True)
    dat.close()
position_df = position_df[position_df['LATITUDE'] > min_lat]

In [ ]:
position_df.head()

In [ ]:

# We load shape files for topographic lines
#first_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_J_1000.shp'
#second_line_path = 'c:\\Users\\flapet\\OneDrive - NOC\\Documents\\NRT_viz\\biocarbon_nrt_data_viz/Data/ne_10m_bathymetry_all/ne_10m_bathymetry_I_2000.shp'

first_line_path  = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_J_1000.shp'
second_line_path = 'C:\\Users\\hanshil\\Documents\\GitHub\\biocarbon_nrt_data_viz\\data\\bathymetry\\ne_10m_bathymetry_I_2000.shp'

gdf_1000 = gpd.read_file(first_line_path)
gdf_2000 = gpd.read_file(second_line_path)

In [ ]:
test1 = xr.open_dataset(os.path.join(NEODASS_dir, 'olci_a_1km-CHL_OC4ME-1d-20240523_20240523-nrt.nc')) # For a and b files
test2 = xr.open_dataset(os.path.join(NEODASS_dir, 'olci_b_1km-CHL_OC4ME-1d-20240523_20240523-nrt.nc'))
print(test1.data_vars)
print(list(test1.keys()))
plot_as_log = True
plotting_lim_max = 1 # Default should be 1?
plotting_lim_min = 0 # Should stay as 0.
log_plotting_lim_max = 20 # Default should be 1?
log_plotting_lim_min = 0.1 # Should stay as 0.
variable_to_plot = 'CHL_OC4ME'

# Combine datasets if both exist
if test1 is not None and test2 is not None:
    # Align the datasets along their dimensions
    test1_aligned, test2_aligned = xr.align(test1, test2, join='outer')
    NEODASS_data_to_plot = test1_aligned.combine_first(test2_aligned)
elif test1 is not None:
    NEODASS_data_to_plot = test1
elif test2 is not None:
    NEODASS_data_to_plot = test2
else:
    raise FileNotFoundError("Neither of the datasets could be found.")


In [ ]:
#Format the date of the loop as a string and use it to set the plot title and filename
date_of_plot    = NEODASS_data_to_plot['time'].data[0]
strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')

if plot_as_log == True:
    chla_log_plot_dir = os.path.join(output_dir, 'Chla_log')
    if os.path.exists(chla_log_plot_dir):
        print(f"Chla log plots saving...")
    else:
        os.makedirs(chla_log_plot_dir)
        print(f"Chla log plots saving...")
    plot_title      = 'Log Chla concentration ' + strdate_of_plot
    plotname        = chla_log_plot_dir + '/' + 'Log_Chla_' + strdate_of_plot + '.png'
else:
    chla_plot_dir = os.path.join(output_dir, 'Chla')
    if os.path.exists(chla_plot_dir):
        print(f"Chla plots saving...")
    else:
        os.makedirs(chla_plot_dir)
        print(f"Chla plots saving...")
    plot_title      = 'Chla concentration ' + strdate_of_plot
    plotname        = chla_plot_dir + '/' + 'Chla_' + strdate_of_plot + '.png'

#Filter the float position data to highlight the last position at the date of the plot
last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

#Set up the plot layout, extent and title
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
ax.set_title(plot_title,fontsize=24)

if plot_as_log == True:
    #Plot the chla colours and the land coastline
    NEODASS_data_to_plot_1day = np.clip(test2[variable_to_plot].data[0,:],log_plotting_lim_min,log_plotting_lim_max+1)
    im_1 = ax.pcolormesh(test1['longitude'].data, test1['latitude'].data, NEODASS_data_to_plot_1day, cmap = color, norm=colors.LogNorm(), transform=ccrs.PlateCarree())
    #im_2 = ax.pcolormesh(test2['longitude'].data, test2['latitude'].data, test2_data_1day, cmap = color, norm=colors.LogNorm(), transform=ccrs.PlateCarree())
    cbar = plt.colorbar(im_1, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')
else:
    #Plot the chla colours and the land coastline
    NEODASS_data_to_plot_1day = np.clip(test2[variable_to_plot].data[0,:],plotting_lim_min,plotting_lim_max+1)
    im_1 = ax.pcolormesh(test1['longitude'].data, test1['latitude'].data, NEODASS_data_to_plot_1day, vmin = plotting_lim_min, vmax = plotting_lim_max, cmap = color, transform=ccrs.PlateCarree())
    #im_2 = ax.pcolormesh(test2['longitude'].data, test2['latitude'].data, test2_data_1day, vmin = plotting_lim_min, vmax = plotting_lim_max, cmap = color, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(im_1, ax = ax, extend = 'max', label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')

ax.add_feature(cfeature.COASTLINE)

#Plot the float position
sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red

#Plot the topo line of 1000 and 2000m depth
gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

#add the lon and lat grid 
gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
gl.top_labels=False   # suppress top labels
gl.right_labels=False # suppress right labels

#save the plot and then close it to avoid high memory usage
plt.savefig(plotname)
plt.clf()
plt.close()

In [ ]:
# Plotting CHLA with a linear colormap.
# define the caracteristics of the map and the features to plot
plotting_chla = True
plotting_lim_max = 1 # Default should be 1?
plotting_lim_min = 0 # Should stay as 0.
if plotting_chla == True :
    chla_plot_dir = os.path.join(output_dir, 'Chla')
    if os.path.exists(chla_plot_dir):
        print(f"Chla plots saving...")
    else:
        os.makedirs(chla_plot_dir)
        print(f"Chla plots saving...")
    for i in tqdm(range(len(chla_data['time'].data))):

        #Trim CHLA data to limits
        chla_data_1day = np.clip(chla_data['CHL'].data[i,:],plotting_lim_min,plotting_lim_max+1)

        #Format the date of the loop as a string and use it to set the plot title and filename
        date_of_plot = chla_data['time'].data[i]
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
        plot_title = 'Chla concentration ' + strdate_of_plot
        plotname = chla_plot_dir + '/' + 'Chla_' + strdate_of_plot + '.png'

        #Filter the float position data to highlight the last position at the date of the plot
        last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
        red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

        #Set up the plot layout, extent and title
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
        ax.set_title(plot_title,fontsize=24)

        #Plot the chla colours and the land coastline
        im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data,
                           chla_data_1day, cmap = color, vmin = plotting_lim_min, vmax = plotting_lim_max,
                           transform=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE)

        #Plot the float position
        sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
        sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red

        #Plot the topo line of 1000 and 2000m depth
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        #add the lon and lat grid 
        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels=False   # suppress top labels
        gl.right_labels=False # suppress right labels

        #format the color bar
        cbar = plt.colorbar(im, ax = ax, extend='max', label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')

        #save the plot and then close it to avoid high memory usage
        plt.savefig(plotname)
        plt.clf()
        plt.close()

In [ ]:
# Plotting Log CHLA
# define the caracteristics of the map and the features to plot
plotting_log = True
plotting_lim_max = 20 # Default should be 1?
plotting_lim_min = 0.1 # Should stay as 0.
if plotting_log == True :
    chla_log_plot_dir = os.path.join(output_dir, 'Chla_log')
    if os.path.exists(chla_log_plot_dir):
        print(f"Chla log plots saving...")
    else:
        os.makedirs(chla_log_plot_dir)
        print(f"Chla log plots saving...")
    for i in tqdm(range(len(chla_data['time'].data))):

        #Format the date of the loop as a string and use it to set the plot title and filename
        date_of_plot = chla_data['time'].data[i]
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
        plot_title = 'Chla concentration ' + strdate_of_plot
        plotname = chla_log_plot_dir + '/' + 'Chla_log_' + strdate_of_plot + '.png'
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')

        #Trim CHLA data to limits
        chla_data_1day  = np.clip(chla_data['CHL'].data[i,:],plotting_lim_min,plotting_lim_max+1)

        #Filter the float position data to highlight the last position at the date of the plot
        last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
        red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

        #Set up the plot layout, extent and title
        fig = plt.figure(figsize=(20, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_title(plot_title,fontsize=24)
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())

        #Plot the chla colours and the land coastline
        im = ax.pcolormesh(chla_data['longitude'].data, chla_data['latitude'].data,
                           chla_data_1day, cmap = color,
                           norm=colors.LogNorm(), transform=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE)

        #Plot the float position
        sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3) # Scatter plot of the previous positions
        sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4) # Scatter plot of the most recent position in red

        #Plot the topo line of 1000 and 2000m depth
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        #add the lon and lat grid 
        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels=False   # suppress top labels
        gl.right_labels=False # suppress right labels

        #format the color bar
        cbar = plt.colorbar(im, ax = ax, label=r'Chlorophyll a surface concentration (mg m$^{-3}$)')

        #save the plot and then close it to avoid high memory usage
        plt.savefig(plotname)
        plt.clf()
        plt.close()

In [ ]:
# Plotting currents
plot_current = True
min_lon = -30 #for the ssh data the extent is lower
if plot_current == True:

    current_plot_dir = os.path.join(output_dir, 'Surface current')
    if os.path.exists(current_plot_dir):
        print(f"Surface current plots saving...")
    else:
        os.makedirs(current_plot_dir)
        print(f"Surface current plots saving...")

    #extract the lon and lat from the dataset only once
    x = current_data['longitude'].data
    y = current_data['latitude'].data

    for i in tqdm(range(len(current_data['time'].data))):

        #From the U and V vector compute the speed, we use it as our colour map
        u = current_data['ugos'].data[i,:]
        v = current_data['vgos'].data[i,:]
        speed = np.sqrt(u**2 + v**2)

        #can be directly added to the quiver function
        vector_crs = ccrs.PlateCarree()

        #Format the date of the loop as a string and use it to make the title and the filename
        date_of_plot = current_data['time'].data[i]
        strdate_of_plot = np.datetime_as_string(date_of_plot, unit='D')
        plot_title = 'Surface current ' + strdate_of_plot
        plotname = current_plot_dir + '/' + 'SSC' + strdate_of_plot + '.png'

        last_pos_df = position_df[position_df['JULD'] <= date_of_plot]
        red_point = last_pos_df[last_pos_df['JULD'] == max(last_pos_df['JULD'])]

        #Set up the plot layout, extent and title
        fig = plt.figure(figsize=(15, 15))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Mercator())
        ax.set_extent([min_lon, max_lon, min_lat, max_lat], crs=ccrs.PlateCarree())
        ax.set_title(plot_title,fontsize=24)

        #Plot the current vectors field and the coastline
        im = ax.quiver(x, y, u, v, speed, angles='xy', scale_units='xy', cmap='viridis', transform=vector_crs, regrid_shape=50, width = 0.002)
        ax.coastlines('50m')

        #Plot the float position
        sc = ax.scatter(last_pos_df['LONGITUDE'], last_pos_df['LATITUDE'], transform=ccrs.PlateCarree(), c = 'black', zorder = 3)
        sc2 = ax.scatter(red_point['LONGITUDE'], red_point['LATITUDE'], transform=ccrs.PlateCarree(), c = 'red', zorder = 4)

        #Plot the topo line of 1000 and 2000m depth
        gdf_1000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.5, edgecolor='k', facecolor='none')
        gdf_2000.plot(ax=ax, transform=ccrs.PlateCarree(), linewidth=0.3, edgecolor='k', facecolor='none')

        #add the lon and lat grid 
        gl = ax.gridlines(draw_labels=True,x_inline=False,y_inline=False, crs=ccrs.PlateCarree())
        gl.top_labels=False   # suppress top labels
        gl.right_labels=False # suppress right labels

        #format the color bar
        cbar = plt.colorbar(im, ax = ax, label=r'Sea surface current (m s$^{-1}$)')
        cbar.set_label(r'Sea surface current (m s$^{-1}$)', rotation=270, labelpad=15)

        #save the plot and then close it to avoid high memory usage
        plt.savefig(plotname)
        plt.clf()
        plt.close()